# v2.3 Forecast on a grid

This notebook is intended test reading in a saved, trained model and deploying it on a grid.


## Environment Setup

In [ ]:
import numpy as np
import sys
sys.path.append('..')
import pickle
import logging
import os.path as osp
import tensorflow as tf
from moisture_rnn_pkl import pkl2train
from moisture_rnn import RNNParams, RNNData, RNN, rnn_data_wrap
from utils import read_yml, read_pkl, retrieve_url, Dict, print_dict_summary, rmse_3d
from moisture_rnn import RNN
import reproducibility
from data_funcs import rmse, to_json, combine_nested, subset_by_features, build_train_dict
from moisture_models import run_augmented_kf
import copy
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import time

## Load Model and Examine

In [ ]:
filename = "../outputs/models/model_predict_raws_rocky.keras"
mod = tf.keras.models.load_model(filename) # prediction model

In [ ]:
# Print Model Summary
mod.summary()

In [ ]:
dat = read_pkl(f"../outputs/models/rnn_data_rocky.pkl")

In [ ]:
type(dat)

In [ ]:
dat.keys()

In [ ]:
dat.X_test.shape

## Predict

In [ ]:
preds = mod.predict(dat.X_test)

print(f"{preds.shape=}")

In [ ]:
errs = rmse_3d(preds, dat.y_test)
print(f"Test Error: {errs}")

## Plot

### Single Plot

In [ ]:
dat.keys()

In [ ]:
from utils import Dict
keys_to_copy = ['features_list', 'all_features_list']
d = Dict({key: dat[key] for key in keys_to_copy if key in dat})

loc_ind = 23

d['id'] = dat.id[loc_ind]
d['X'] = dat.X[loc_ind][dat.test_ind:, :]
d['y'] = dat.y_test[loc_ind, :, :]
d['hours'] = len(d['y'])
d['m'] = preds[loc_ind, :, :]

In [ ]:
import importlib
import data_funcs
importlib.reload(data_funcs)
from data_funcs import plot_data

In [ ]:
plot_data(d, title="RNN Prediction Error", title2=d['id'], plot_period="all")

## Plot Grid

In [ ]:
dat.loc.keys()

In [ ]:
test_indices = [i for i, val in enumerate(dat["id"]) if val in dat.loc['test_locs']]
x_coord_test = [dat.loc["pixel_x"][i] for i in test_indices]
y_coord_test = [dat.loc["pixel_y"][i] for i in test_indices]

In [ ]:
len(x_coord_test)

In [ ]:
preds.shape

In [ ]:
tstep = 0

sc = plt.scatter(
    x_coord_test, y_coord_test,
    c = preds[:, tstep, 0],  # Assuming the first dimension is to be used for color
    cmap='viridis'  # Choose a colormap
)
plt.colorbar(sc)  # Add colorbar for scale
plt.xlabel("X Grid Coordinate")
plt.ylabel("Y Grid Coordinate")
plt.show()

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib.animation import FuncAnimation, PillowWriter


fig, ax = plt.subplots()
sc = ax.scatter(x_coord_test, y_coord_test, c=preds[:, 0, 0], cmap='viridis')
plt.colorbar(sc)
ax.set_xlabel("X Grid Coordinate")
ax.set_ylabel("Y Grid Coordinate")

# Function to update the scatter plot at each frame
def update(tstep):
    sc.set_array(preds[:, tstep, 0])  # Update the colors based on tstep
    ax.set_title(f'Time Step: {tstep}')
    return sc,

# Number of frames (time steps)
num_timesteps = preds.shape[1]

# Create the animation
ani = FuncAnimation(
    fig, update, frames=np.arange(num_timesteps), interval=200, repeat=True
)

# Save the animation as a GIF
ani.save("../outputs/animation.gif", writer=PillowWriter(fps=5))


In [ ]:
# from IPython.display import Image
# Image(filename="../outputs/animation.gif")